***Importing data***

In [120]:
import pandas as pd
exoTrain = pd.read_csv("exoTrain.csv")

exoTrain["LABEL"] = exoTrain["LABEL"].replace(1, 0)
exoTrain["LABEL"] = exoTrain["LABEL"].replace(2, 1)

exoTrain['star_id'] = range(1, len(exoTrain) + 1)
exoTrain['star_id'] = exoTrain['star_id'].astype('str')

***TIME SERIES FORECASTING USING FACEBOOK'S PROPHET MODEL*** 🔮

*A statistical method of analyzing historical time data to make predictions on future values based on previous observations.*

***Data preparation for training our model***

In [23]:
exoTrain_T = exoTrain.T 

#naming the column names as star ids and deleting the id row from data frame
exoTrain_T.columns = exoTrain_T.iloc[3198]
exoTrain_T.drop(exoTrain_T.tail(1).index,inplace=True) 
print(exoTrain_T.tail(5))
#exoTrain_T.head(5)

star_id        1      2      3      4       5      6      7       8      9  \
FLUX.3193  92.54   0.76   5.06 -12.67 -438.54  19.27  -0.44    95.3   4.53   
FLUX.3194  39.32  -11.7  -11.8  -8.77 -399.71  -43.9   10.9   48.86  21.95   
FLUX.3195  61.42   6.46 -28.91 -17.31 -384.65 -41.63 -11.77  -10.62  26.94   
FLUX.3196   5.08   16.0 -70.02 -17.35 -411.79  -52.9  -9.25 -112.02  34.08   
FLUX.3197 -39.54  19.93 -96.67  13.98 -510.54 -16.16 -36.69 -229.92  44.65   

star_id         10  ...    5078   5079    5080   5081   5082    5083    5084  \
FLUX.3193  3162.53  ...  210.09   8.75  163.02  28.82 -74.95  151.75 -136.16   
FLUX.3194  3398.28  ...     3.8 -10.69   86.29 -20.12 -46.29  -24.45   38.03   
FLUX.3195  3648.34  ...   16.33  -9.54   13.06 -14.41  -3.08   -17.0  100.28   
FLUX.3196  3671.97  ...   27.35  -2.48  161.22 -43.35 -28.43    3.23  -45.64   
FLUX.3197  3781.91  ...    21.3  -8.69   213.6 -30.04 -48.68   19.28   35.58   

star_id     5085   5086   5087  
FLUX.3193  -3.47 

In [24]:
exoTrain_T.columns = exoTrain_T.columns.astype(str)
exoTrain_T.columns = exoTrain_T.columns.str.replace('.0', '')
exoTrain_T.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


star_id,1,2,3,4,5,6,7,8,9,,...,78,79,,81,82,83,84,85,86,87
LABEL,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
FLUX.1,93.85,-38.88,532.64,326.52,-1107.21,211.1,9.34,238.77,-103.54,-265.91,...,125.57,7.45,475.61,-46.63,299.41,-91.91,989.75,273.39,3.82,323.28
FLUX.2,83.81,-33.83,535.92,347.39,-1112.59,163.57,49.96,262.16,-118.97,-318.59,...,78.69,10.02,395.5,-55.39,302.77,-92.97,891.01,278.0,2.09,306.36
FLUX.3,20.1,-58.54,513.73,302.35,-1118.95,179.16,33.3,277.8,-108.93,-335.66,...,98.29,6.87,423.61,-64.88,278.68,-78.76,908.53,261.73,-3.29,293.16
FLUX.4,-26.98,-40.09,496.92,298.13,-1095.1,187.82,9.63,190.16,-72.25,-450.47,...,91.16,-2.82,376.36,-88.75,263.48,-97.33,851.83,236.99,-2.88,287.67


In [25]:
exoTrain_T.drop(exoTrain_T.head(1).index,inplace=True)
exoTrain_T.head(5)

star_id,1,2,3,4,5,6,7,8,9,,...,78,79,,81,82,83,84,85,86,87
FLUX.1,93.85,-38.88,532.64,326.52,-1107.21,211.1,9.34,238.77,-103.54,-265.91,...,125.57,7.45,475.61,-46.63,299.41,-91.91,989.75,273.39,3.82,323.28
FLUX.2,83.81,-33.83,535.92,347.39,-1112.59,163.57,49.96,262.16,-118.97,-318.59,...,78.69,10.02,395.5,-55.39,302.77,-92.97,891.01,278.0,2.09,306.36
FLUX.3,20.1,-58.54,513.73,302.35,-1118.95,179.16,33.3,277.8,-108.93,-335.66,...,98.29,6.87,423.61,-64.88,278.68,-78.76,908.53,261.73,-3.29,293.16
FLUX.4,-26.98,-40.09,496.92,298.13,-1095.1,187.82,9.63,190.16,-72.25,-450.47,...,91.16,-2.82,376.36,-88.75,263.48,-97.33,851.83,236.99,-2.88,287.67
FLUX.5,-39.56,-79.31,456.45,317.74,-1057.55,188.46,37.64,180.98,-61.46,-453.09,...,78.42,-1.56,338.94,-75.4,236.89,-68.0,755.11,280.73,1.66,249.89


In order for our prophet forecasting model to work, the dataframe must contain *ds* and *y* columns containing date and values respectively. 

But, our ds column contains string entries.

`ParserError: Unknown string format: FLUX.1`

Hence, for the purpose of training our model for our data, we convert this column entries to arbitrary datetime values. 

They can be translated to FLUX values later on.

*The datasets were prepared late-summer 2016.*

So, we create a new column of dates starting from August 15, 2016 to replace the flux entries for the purpose of fitting the data in our time-series forecasting model.

In [26]:
exoTrain_T['date'] = pd.date_range(start='15/08/2016', periods=len(exoTrain_T), freq='D')
#exoTrain_T.head(5)
exoTrain_T.tail(5)

star_id,1,2,3,4,5,6,7,8,9,,...,79,,81,82,83,84,85,86,87,date
FLUX.3193,92.54,0.76,5.06,-12.67,-438.54,19.27,-0.44,95.3,4.53,3162.53,...,8.75,163.02,28.82,-74.95,151.75,-136.16,-3.47,-1.5,-25.33,2025-05-12
FLUX.3194,39.32,-11.7,-11.8,-8.77,-399.71,-43.9,10.9,48.86,21.95,3398.28,...,-10.69,86.29,-20.12,-46.29,-24.45,38.03,65.73,-4.65,-41.31,2025-05-13
FLUX.3195,61.42,6.46,-28.91,-17.31,-384.65,-41.63,-11.77,-10.62,26.94,3648.34,...,-9.54,13.06,-14.41,-3.08,-17.0,100.28,88.42,-14.55,-16.72,2025-05-14
FLUX.3196,5.08,16.0,-70.02,-17.35,-411.79,-52.9,-9.25,-112.02,34.08,3671.97,...,-2.48,161.22,-43.35,-28.43,3.23,-45.64,79.07,-6.41,-14.09,2025-05-15
FLUX.3197,-39.54,19.93,-96.67,13.98,-510.54,-16.16,-36.69,-229.92,44.65,3781.91,...,-8.69,213.6,-30.04,-48.68,19.28,35.58,79.43,-2.55,27.82,2025-05-16


In [27]:
#exoTrain_T = exoTrain_T.reset_index()

df = pd.melt(exoTrain_T, id_vars = 'date', value_vars = exoTrain_T.iloc[:,0:5087])
df.columns = ['ds', 'star_id', 'y'] #df.columns = ['time_point', 'star_id', 'light_intensity']
df.tail(3198) #checking if we have all the flux entries 

,ds,star_id,y
16259941,2025-05-16,4998,-6.27
16259942,2016-08-15,4999,-26.68
16259943,2016-08-16,4999,-22.69
16259944,2016-08-17,4999,-20.56
16259945,2016-08-18,4999,-26.62
...,...,...,...
16263134,2025-05-12,4999,-42.82
16263135,2025-05-13,4999,3.46
16263136,2025-05-14,4999,11.78
16263137,2025-05-15,4999,3.45


In [28]:
len(df['star_id'].unique())

3750

In [29]:
df.shape

(16263139, 3)

After transforming the data from the wide format to the long format, we have 16263139 records.

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16263139 entries, 0 to 16263138
Data columns (total 3 columns):
 #   Column   Dtype         
---  ------   -----         
 0   ds       datetime64[ns]
 1   star_id  object        
 2   y        object        
dtypes: datetime64[ns](1), object(2)
memory usage: 372.2+ MB


Next, we group the pandas dataframe by the column *star_id* and save it in a new dataframe called *groups_by_star_id*. 

In [31]:
#group data by star_id
groups_by_star_id = df.groupby('star_id')

#check groups in the dataframe
len(groups_by_star_id.groups.keys())

3750

***Writing a function to train and forecast each group***

After predicting the future dataframe, Prophet produces a long list of outputs.

We only kept ds, yhat, yhat_lower and yhat_upper.

*   yhat is the predicted value.

*   yhat_lower and yhat_upper are the lower and upper bound of the uncertainty interval.


In [32]:
from prophet import Prophet

def train_and_forecast(group):

  #initiate the model
  m = Prophet()

  #fit the model
  m.fit(group)

  #make predictions
  future = m.make_future_dataframe(periods = 15) #period=15 means that the function predicts the next 15 light intensity values for each star_id.
  forecast = m.predict(future)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']] #  forecast = m.predict(future)[['time_point', 'yhat', 'yhat_lower', 'yhat_upper']]
  forecast['star_id'] = group['star_id'].iloc[0]

  #return forecasted results
  return forecast[['ds', 'star_id', 'yhat', 'yhat_upper', 'yhat_lower']]

***Multiple time series forecasting using for-loop***

```
star_id_list = (df["star_id"].unique()).tolist()

#create an empty dataframe
for_loop_forecast = pd.DataFrame()

#loop through each star_id
for star_id in star_id_list:
  #get the data for each star_id
  group = groups_by_star_id.get_group(star_id)
  #make forecast
  forecast = train_and_forecast(group)
  #add forecast results to the dataframe
  for_loop_forecast = pd.concat((for_loop_forecast, forecast))

for_loop_forecast.head(5)
```

Performing multiple time series forecasting by for-looping was taking an eternity 👵🏻

So, we use Python's `mutiprocessing` package to run the forecasts in parallel.

***Multiple time series forecasting using Multi-Processing***

In [35]:
import multiprocessing

#get time series data for each star_id and save in a list
series = [groups_by_star_id.get_group(star_id) for star_id in star_id_list]

#create a pool process with the number of worker processes being the number of CPUs
from multiprocessing import Pool
import os
p = Pool(os.cpu_count())

#make predictions for each star_id and save the result to a list
from tqdm import tqdm
import time
predictions = list(tqdm(p.imap(train_and_forecast, series), total=len(series)))

#terminate the pool process
p.close()

#tell the pool to wait till all the jobs are finished before exit
p.join()

#concatenate results
multiprocess_forecast = pd.concat(predictions)

Streaming output truncated to the last 5000 lines.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5oal43fk/1hgca_5v.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94100', 'data', 'file=/tmp/tmp5oal43fk/ovs_c0ok.json', 'init=/tmp/tmp5oal43fk/1hgca_5v.json', 'output', 'file=/tmp/tmp5oal43fk/prophet_modelx9fbnpul/prophet_model-20221015095758.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:57:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:57:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
 87%|████████▋ | 3250/3750 [2:33:56<16:55,  2.03s/it]INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5oal43fk/6558k6uk.json
DEBUG:cmdstanpy:input tempfile: /tmp/tm

Thank you for patiently scrolling 👵🏻

Really appreciate it 🤍

In [36]:
type(multiprocess_forecast)

pandas.core.frame.DataFrame

In [83]:
multiprocess_forecast.head(5)

,ds,star_id,yhat,yhat_upper,yhat_lower
0,2016-08-15,1,-13.755567,119.634607,-162.458141
1,2016-08-16,1,-13.441980,127.379265,-152.266148
2,2016-08-17,1,-13.637111,124.952621,-145.475576
3,2016-08-18,1,-13.917658,122.046222,-150.452084
4,2016-08-19,1,-13.820799,125.450520,-158.616738


***Exploring forecasted predictions***

In [84]:
(multiprocess_forecast['star_id']).unique()

array(['1', '2', '3', ..., '4997', '4998', '4999'], dtype=object)

In [85]:
star_1_forecast = multiprocess_forecast.loc[(multiprocess_forecast['star_id'] == '1')]

***Converting the arbitrary date-time values back to 'FLUX.XXXX' values***

In [86]:
star_1_forecast['row_1'] = 'FLUX.'

star_1_forecast['row_2'] = range(1, len(star_1_forecast) + 1)
star_1_forecast['row_2'] = star_1_forecast['row_2'].astype('str')

star_1_forecast['flux'] = star_1_forecast['row_1'] + star_1_forecast['row_2']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [88]:
star_1_forecast.drop(['ds', 'row_1', 'row_2'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [117]:
star_1_forecast.head(5)

,star_id,yhat,yhat_upper,yhat_lower,flux
0,1,-13.755567,119.634607,-162.458141,FLUX.1
1,1,-13.441980,127.379265,-152.266148,FLUX.2
2,1,-13.637111,124.952621,-145.475576,FLUX.3
3,1,-13.917658,122.046222,-150.452084,FLUX.4
4,1,-13.820799,125.450520,-158.616738,FLUX.5


In [90]:
star_1_forecast.star_id.unique()

array(['1'], dtype=object)

***Plotting Prophet's future flux values of an exoplanet - star 1***

In [116]:
import plotly.graph_objects as px
import numpy as np

plot = px.Figure(data=[px.Scatter(
	x=star_1_forecast['flux'],
	y=star_1_forecast['yhat'], 
	line=dict(color="darkturquoise"),
	mode='lines',)
])

plot.show()

***Plotting Prophet's future flux values of an non-exoplanet - star 38***

Let's select a non-exoplanet to plot its forecasted flux predictions!

In [123]:
non_exo = exoTrain.loc[(exoTrain['LABEL'] == 0)]
non_exo.head(1)

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197,star_id
37,0,-141.22,-81.79,-52.28,-32.45,-1.55,-35.61,-23.28,19.45,53.11,...,-22.34,-36.23,27.44,13.52,38.66,-17.53,31.49,31.38,50.03,38


In [133]:
star_38_forecast = multiprocess_forecast.loc[(multiprocess_forecast['star_id'] == '38')]

In [134]:
star_38_forecast['row_1'] = 'FLUX.'

star_38_forecast['row_2'] = range(1, len(star_38_forecast) + 1)
star_38_forecast['row_2'] = star_38_forecast['row_2'].astype('str')

star_38_forecast['flux'] = star_38_forecast['row_1'] + star_38_forecast['row_2']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [135]:
star_38_forecast.drop(['ds', 'row_1', 'row_2'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [136]:
plot = px.Figure(data=[px.Scatter(
	x=star_38_forecast['flux'],
	y=star_38_forecast['yhat'], 
	line=dict(color="red"),
	mode='lines',)
])

plot.show()